In [ ]:
# We fine-tune a pretrained transformer encoder to perform aspect-based emotion classification.”
"""
Modeling Rationale

Although classical machine learning models such as Logistic Regression or
Support Vector Machines can be trained from scratch for text classification,
they rely on shallow lexical representations (e.g., bag-of-words or TF–IDF)
and therefore lack an explicit notion of context, word order, and semantic
composition.

Emotion classification, and especially aspect-based emotion classification,
is inherently context-dependent. The same sentence may express different
emotions depending on the referenced aspect, and emotional meaning is often
conveyed implicitly through contrast, expectation, or discourse structure
rather than through isolated keywords.

Training a model from scratch under these conditions would require learning
both linguistic structure and emotional semantics simultaneously, which is
impractical with limited labeled data. Instead, we fine-tune a pretrained
transformer encoder that already captures rich syntactic and semantic
regularities of natural language. Fine-tuning allows the model to adapt this
general linguistic knowledge to the specific task of emotion classification
with substantially fewer examples and improved generalization.

For these reasons, a fine-tuned transformer-based classifier is more suitable
than classical models trained from scratch for the task considered here.
"""

In [ ]:
"""
Model Choice Justification

We use DistilRoBERTa as the underlying language encoder for emotion classification.
DistilRoBERTa is a distilled version of RoBERTa that retains most of its
representational power while being significantly smaller and faster. This makes
it well-suited for experimental settings with limited computational resources
and moderate-sized datasets, while still providing strong contextual language
understanding.

RoBERTa-based models are particularly effective for sentiment and emotion-related
tasks because they are pretrained on large amounts of diverse text using a
masked language modeling objective that captures nuanced lexical, syntactic,
and semantic relationships. These properties are critical for emotion detection,
where affect is often expressed implicitly rather than through explicit emotion
words.

DistilRoBERTa offers a practical trade-off between performance and efficiency:
it enables stable fine-tuning, faster iteration, and reduced overfitting risk
compared to larger models, without sacrificing the core benefits of contextual
representations.

Alternative pretrained models could also be used within the same framework.
For example:
- BERT-base can serve as a widely adopted baseline, though it is generally
  weaker than RoBERTa-style models on sentiment-oriented tasks.
- RoBERTa-base may yield slightly higher performance at the cost of increased
  memory usage and slower training.
- ALBERT provides parameter sharing for efficiency but can be less stable during
  fine-tuning.
- Domain-specific models (e.g., sentiment-adapted or review-trained transformers)
  may further improve performance if available.

The choice of DistilRoBERTa therefore reflects a deliberate balance between model
capacity, training stability, computational efficiency, and suitability for
aspect-based emotion classification.
"""

In [ ]:
import os
import sys

PROJECT_MARKERS = ("src", "data", "prompts", "results")

def find_project_root(start_path):
    current = os.path.abspath(start_path)

    while True:
        if all(os.path.isdir(os.path.join(current, m)) for m in PROJECT_MARKERS):
            return current

        parent = os.path.dirname(current)
        if parent == current:
            raise RuntimeError("Project root not found")

        current = parent


# ---- execution directory (cwd) ----
cwd = os.getcwd()

# ---- safe starting point ----
try:
    start_path = os.path.dirname(os.path.abspath(__file__))
except NameError:
    start_path = cwd


# ---- resolve canonical paths ----
project_root = find_project_root(start_path)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

src_root     = os.path.join(project_root, "src", "daniel", "gemini")
data_root    = os.path.join(project_root, "data", "MAMS-ACSA", "raw", "data_jsonl")
schemas_root = os.path.join(project_root, "data", "MAMS-ACSA", "raw", "data_jsonl", "schema")
prompts_root = os.path.join(project_root, "prompts", "daniel", "llama")
utils_root   = os.path.join(project_root, "utils")
results_root = os.path.join(project_root, "results", "daniel","training_results")

print(
    f"cwd          : {cwd}\n"
    f"Project root : {project_root}\n"
    f"Source root  : {src_root}\n"
    f"Data root    : {data_root}\n"
    f"Prompts root : {prompts_root}\n"
    f"Utils root   : {utils_root}\n"
    f"Results root : {results_root}"
)

In [ ]:
import json
from pathlib import Path

import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

from transformers import (
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments
)

In [ ]:
def load_jsonl(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            rows.append(json.loads(line))
    return rows

train_rows = load_jsonl(Path(data_root) / "train.jsonl")
val_rows   = load_jsonl(Path(data_root) / "val.jsonl")
test_rows  = load_jsonl(Path(data_root) / "test.jsonl")

In [5]:
def light_check(rows, name):
    for i, r in enumerate(rows):
        assert isinstance(r, dict), f"{name}[{i}] is not a dict"

        # input
        assert "input" in r, f"{name}[{i}] missing 'input'"
        assert isinstance(r["input"], str), f"{name}[{i}]['input'] not a string"
        assert r["input"].strip(), f"{name}[{i}] empty 'input'"

        # output
        assert "output" in r, f"{name}[{i}] missing 'output'"
        assert isinstance(r["output"], list), f"{name}[{i}]['output'] not a list"
        assert len(r["output"]) > 0, f"{name}[{i}] empty 'output' list"

        # # each label item
        # for j, o in enumerate(r["output"]):
        #     assert isinstance(o, dict), f"{name}[{i}]['output'][{j}] not a dict"
        #     for k in ("aspect", "polarity", "emotion"):
        #         assert k in o, f"{name}[{i}]['output'][{j}] missing '{k}'"
        #         assert isinstance(o[k], str), f"{name}[{i}]['output'][{j}]['{k}'] not a string"
        #         assert o[k].strip(), f"{name}[{i}]['output'][{j}] empty '{k}'"

    print(f"{name}: {len(rows)} rows passed")


light_check(train_rows, "train")
light_check(val_rows, "val")
light_check(test_rows, "test")

train: 3149 rows passed
val: 400 rows passed
test: 400 rows passed


In [ ]:
def explode(rows):
    records = []
    for r in rows:
        text = r["input"]
        for o in r["output"]:
            records.append({
                "text": text,
                "aspect": o["aspect"],
                "polarity": o["polarity"],
                "emotion": o["emotion"]
                
            })
    return pd.DataFrame(records)

In [ ]:
train_df = explode(train_rows)
val_df   = explode(val_rows)
test_df  = explode(test_rows)

len(train_df), len(val_df), len(test_df)

In [ ]:
for df in (train_df, val_df, test_df):
    df["emotion"] = df["emotion"].replace({"mentioned_only": "neutral"})

In [ ]:
train_df["emotion"].value_counts()

In [ ]:
emotion_encoder = LabelEncoder()
polarity_encoder = LabelEncoder()

# fit on TRAIN only
train_df["emotion_id"]  = emotion_encoder.fit_transform(train_df["emotion"])
train_df["polarity_id"] = polarity_encoder.fit_transform(train_df["polarity"])

# apply to VAL / TEST
val_df["emotion_id"]  = emotion_encoder.transform(val_df["emotion"])
val_df["polarity_id"] = polarity_encoder.transform(val_df["polarity"])

test_df["emotion_id"]  = emotion_encoder.transform(test_df["emotion"])
test_df["polarity_id"] = polarity_encoder.transform(test_df["polarity"])

emotion_encoder.classes_, polarity_encoder.classes_

In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=256):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]

        text = f"ASPECT: {row['aspect']} | TEXT: {row['text']}"

        enc = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "emotion_labels": torch.tensor(row["emotion_id"]),
            "polarity_labels": torch.tensor(row["polarity_id"]),
        }

In [ ]:
MODEL_NAME = "distilroberta-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_ds = EmotionDataset(train_df, tokenizer)
val_ds   = EmotionDataset(val_df, tokenizer)
test_ds  = EmotionDataset(test_df, tokenizer)

len(train_ds), len(val_ds), len(test_ds)

In [ ]:
class EmotionPolarityModel(torch.nn.Module):
    def __init__(self, model_name, num_emotions, num_polarity):
        super().__init__()

        self.encoder = AutoModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.hidden_size

        self.emotion_head  = torch.nn.Linear(hidden_size, num_emotions)
        self.polarity_head = torch.nn.Linear(hidden_size, num_polarity)

    def forward(
        self,
        input_ids,
        attention_mask,
        emotion_labels=None,
        polarity_labels=None
    ):
        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        cls_repr = outputs.last_hidden_state[:, 0]

        emotion_logits  = self.emotion_head(cls_repr)
        polarity_logits = self.polarity_head(cls_repr)

        loss = None
        if emotion_labels is not None:
            loss_emotion = torch.nn.functional.cross_entropy(
                emotion_logits, emotion_labels
            )
            loss_polarity = torch.nn.functional.cross_entropy(
                polarity_logits, polarity_labels
            )
            loss = loss_emotion + 0.3 * loss_polarity

        return {
            "loss": loss,
            "emotion_logits": emotion_logits,
            "polarity_logits": polarity_logits,
        }

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    emotion_logits = logits["emotion_logits"]
    emotion_labels = labels["emotion_labels"]

    preds = np.argmax(emotion_logits, axis=1)

    return {
        "emotion_f1_macro": f1_score(
            emotion_labels,
            preds,
            average="macro"
        )
    }

In [ ]:
model = EmotionPolarityModel(
    model_name=MODEL_NAME,
    num_emotions=len(emotion_encoder.classes_),
    num_polarity=len(polarity_encoder.classes_)
)

training_args = TrainingArguments(
    output_dir=results_root,
    evaluation_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()